# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,302.508453,-0.696691,1234,301.811762,303.205144,603.623524,606.410289
6,630.904893,0.659162,1234,631.564055,630.245731,1263.128110,1260.491462
7,942.063456,-0.866896,1234,941.196560,942.930352,1882.393121,1885.860703
8,37.892562,-0.055831,1234,37.836731,37.948393,75.673461,75.896786
9,855.386358,-0.725296,1234,854.661062,856.111654,1709.322124,1712.223308
10,615.363061,-0.316747,1234,615.046314,615.679808,1230.092629,1231.359616
11,852.296955,0.691898,1234,852.988853,851.605057,1705.977707,1703.210113
12,135.465279,-0.743617,1234,134.721662,136.208896,269.443324,272.417792
13,748.799264,0.206665,1234,749.005929,748.592599,1498.011857,1497.185198
14,39.164748,0.017628,1234,39.182376,39.147120,78.364751,78.294240


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-301.811762,303.205144
6,-631.564055,630.245731
7,-941.196560,942.930352
8,-37.836731,37.948393
9,-854.661062,856.111654
10,-615.046314,615.679808
11,-852.988853,851.605057
12,-134.721662,136.208896
13,-749.005929,748.592599
14,-39.182376,39.147120


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-301.811762,303.205144
6,-631.564055,630.245731
7,-941.196560,942.930352
8,-37.836731,37.948393
9,-854.661062,856.111654
10,-615.046314,615.679808
11,-852.988853,851.605057
12,-134.721662,136.208896
13,-749.005929,748.592599
14,-39.182376,39.147120


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-301.8117619 ,  303.2051444 ],
       [-631.56405476,  630.24573115],
       [-941.19656043,  942.93035158],
       [ -37.83673073,   37.94839312],
       [-854.66106221,  856.11165375],
       [-615.04631429,  615.67980814],
       [-852.98885349,  851.60505663],
       [-134.72166194,  136.20889613],
       [-749.00592867,  748.592599  ],
       [ -39.18237573,   39.14711988]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,601.623524,-606.410289
6,1261.128110,-1260.491462
7,1880.393121,-1885.860703
8,73.673461,-75.896786
9,1707.322124,-1712.223308
10,1228.092629,-1231.359616
11,1703.977707,-1703.210113
12,267.443324,-272.417792
13,1496.011857,-1497.185198
14,76.364751,-78.294240


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
